In [ ]:
!pip install flask
from flask import Flask, jsonify, request
import sqlite3


In [ ]:
app = Flask(__name__)
db_path = '/content/student.db'  # Set the path for the SQLite database file

# Create the student table in the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS students (
        student_id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        dob TEXT NOT NULL,
        amount_due REAL
    )
''')
conn.commit()
conn.close()


In [ ]:
# Create a new student record
@app.route('/students', methods=['POST'])
def create_student():
    student_data = request.get_json()
    first_name = student_data['first_name']
    last_name = student_data['last_name']
    dob = student_data['dob']
    amount_due = student_data['amount_due']

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('INSERT INTO students (first_name, last_name, dob, amount_due) VALUES (?, ?, ?, ?)',
                   (first_name, last_name, dob, amount_due))
    conn.commit()
    conn.close()

    return jsonify({'message': 'Student record created successfully'}), 201

# Get a specific student record by student_id
@app.route('/students/<int:student_id>', methods=['GET'])
def get_student(student_id):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM students WHERE student_id = ?', (student_id,))
    student = cursor.fetchone()
    conn.close()

    if student is None:
        return jsonify({'message': 'Student not found'}), 404

    student_data = {
        'student_id': student[0],
        'first_name': student[1],
        'last_name': student[2],
        'dob': student[3],
        'amount_due': student[4]
    }

    return jsonify(student_data), 200

# Update an existing student record
@app.route('/students/<int:student_id>', methods=['PUT'])
def update_student(student_id):
    student_data = request.get_json()
    first_name = student_data['Ray']
    last_name = student_data['Wong']
    dob = student_data['9-June-2000']
    amount_due = student_data['199.99']

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('UPDATE students SET first_name = ?, last_name = ?, dob = ?, amount_due = ? WHERE student_id = ?',
                   (first_name, last_name, dob, amount_due, student_id))
    conn.commit()
    conn.close()

    return jsonify({'message': 'Student record updated successfully'}), 200

# Delete a student record
@app.route('/students/<int:student_id>', methods=['DELETE'])
def delete_student(student_id):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('DELETE FROM students WHERE student_id = ?', (student_id,))
    conn.commit()
    conn.close()

    return jsonify({'message': 'Student record deleted successfully'}), 200

# Get all student records
@app.route('/students', methods=['GET'])
def get_all_students():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM students')
    students = cursor.fetchall()
    conn.close()

    student_list = []
    for student in students:
        student_data = {
            'student_id': student[0],
            'first_name': student[1],
            'last_name': student[2],
            'dob': student[3],
            'amount_due': student[4]
        }
        student_list.append(student_data)

    return jsonify(student_list), 200


In [ ]:
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
